# Parquet generator and streamer

In [42]:
%run config.py

In [ ]:
BATCH_SIZE = 1024
TARGET_PATH = DATA_PATH
FILE_NAME = 'test_arthur'
OUTPUT_STREAM_PATH = STREAM_CONFIGS['generated-stream']['path']
SHARDS_COUNT = STREAM_CONFIGS['generated-stream']['shard_count']
SHARD_KEY = PARTITION_ATTR

In [ ]:
!python -m pip install pandas
!python -m pip install pyarrow
!python -m pip install fastparquet
!python -m pip install v3io


# Run to generate gzip parquet file

In [ ]:
"""
event_dict = [{"user_id" : str(i) , "event_type": "spin" + str(i)} for i in range(10000000)]
file_name = "test_arthur"
target_path = os.getenv('TARGET_PATH')
df = pd.DataFrame.from_records(event_dict)
df.to_parquet(path=os.path.join(target_path, file_name), compression='gzip')
"""

# Stream Parquet to Output Stream Path

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import datetime
import fastparquet
import v3io.dataplane


In [ ]:
def handler(event):
    V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
    v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=V3IO_ACCESS_KEY)
    if type(event) is dict:
        event_dict = event
    else:
        event_dict = json.loads(event)
    parquet = event_dict["file"]
    partition_key = event_dict["partition_key"] or None
    pfile = fastparquet.ParquetFile(parquet)
    for i, df in enumerate(pfile.iter_row_groups()):
        write_header = (i==0)
        n = 1000  #chunk row size
        list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
        for frame in list_df:
            records = []

            for n, row in frame.iterrows():
                records.append(event_to_record(row.to_dict(),partition_key))
            resp = v3io_client.put_records(container=CONTAINER, 
                                           path=OUTPUT_STREAM_PATH, 
                                           records=records, 
                                           raise_for_status=v3io.dataplane.RaiseForStatus.never)
            
def event_to_record(event_dict, partition_key=None):
    event_str = json.dumps(event_dict)
    return {'data': event_str, 'partition_key': str(partition_key)}



## Test Locally

In [ ]:
path = FILE_PATH
event = {"file" :  path, "partition_key" : None}
handler(event)